
# Introduction to the Simulation

In this Notebook, the simulation method from the paper is explained on the exemplary railway junction from the paper.

First of all, we import the needed modules. For the actual simulation, we use the python package simpy.

In [ ]:
import time
import simpy

import sys
import os

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.Simulator import Simulator
from src.JunctionContainer import JunctionContainer, TrainMixContainer
from src.CorrectnessTests import CorrectnessTests
start = time.time()

We start by creating a new environment.

In [ ]:
env = simpy.Environment()

Furthermore, the exemplary junction is build. In this case we use an artificial train mix with 6 high-speed passenger trains per hour on all four routes. (Note that we assume the same traffic in both directions and therefore, only the traffic for the main line and the branching line needs to be given) 

In [ ]:
junction = JunctionContainer(TrainMixContainer(6, 0, 0, 0), TrainMixContainer(6, 0, 0, 0), 't', 't2')
junction.time_frame = 60

Therefore, we model arrival rates of

In [ ]:
print(f'Arrrival rate main lines: {junction.get_arrival_rate_main_branch():.4f} ')
print(f'Arrrival rate branching lines: {junction.get_arrival_rate_side_branch():.4f} ')

in this example. Feel free to change the train mix to get a feeling for the model.

Regarding the service rate, we opt to fix it at 0.3 for every route, in correspondence with the simulations conducted in the paper. 

In [ ]:
route_service_rate = {
    'a-b': 0.3,
    'a-c': 0.3,
    'b-a': 0.3,
    'c-a': 0.3
}

Now we are setup to start a simulation run. This can be done for any number of time_steps. We use 1320 minutes here, which would correspond to 22 hours.

In [ ]:
setup = time.time()
a = Simulator.run_junction_sim(env, junction, route_service_rate, run_until=1320)
sim = time.time()

The simulation took

In [ ]:
print(f'{(sim - setup):.4f} s.')

We can now analyse the queue-lengths from the simulation log, which took a snapshot of the queue length for every route in every minute. To further compare this with the analytical results, we can build the average over the simulated time span. To allow the simulation some time for starting and cooling off, we use only the minutes 60-1260, therefore 20 hours, for this analysis.

In [ ]:
print({k: r.calc_length_of_queue(start=60, end=1260) for k, r in a.routes.items()})

The conflict-freeness of the simulation can be tested with the method 'eval_overlapping_conflicts', that counts the number of minutes, in which both routes of every route-pair have been occupied and adds up all the occurences of double occupation of conflicting routes. 

In [ ]:
sum_of_conflicts = CorrectnessTests.eval_overlapping_conflicts(a)
print(f'Number of conflicts: {sum_of_conflicts}')

In the validation part, this simulation has been performed for different numbers of trains and traffic shares of main and branching line. To facilitate for the random effects in the simulation, 100 simulations have been performed for every parameter setting and combined by averaging over all queue-lenghts. Furthermore, in addition to the results, the number of conflicts has been stored for every simulation run, ensuring the correctness of all simulation runs.

Feel free to adapt this code and use it for your own simulations, including a reference to the paper would be highly appreciated.

Cheers!